In [1]:
%%capture
!pip install skorch

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import os
import warnings
warnings.simplefilter('ignore')
 
import torch.nn.functional as F
from torch import nn
import torch
 
from skorch import NeuralNetClassifier
from skorch.callbacks import Callback, Checkpoint, EarlyStopping
from skorch.dataset import CVSplit

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split

from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def torch_fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

torch_fix_seed()

In [4]:
INPUT_PATH = "/content/drive/MyDrive/GCI/titanic/Input"

train = pd.read_csv(os.path.join(INPUT_PATH, "train.csv"))
test = pd.read_csv(os.path.join(INPUT_PATH, "test.csv"))
train["Perished"] = train["Perished"].replace({1:0, 0:1})
test["Perished"] = 0

PassengerId = test["PassengerId"]
data = pd.concat([train, test], ignore_index = True)

# 特徴量作成

In [5]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data["Perished"] = data["Perished"].astype('int64')

data["Ticket_Freq"] = data["Ticket"].str.len()
data["Title"] =  data["Name"].apply(lambda x:x.split(',')[1].split('.')[0].strip())
data["Surname"] = data["Name"].apply(lambda x:x.split(',')[0])

data["Fare"] = data["Fare"].fillna(data["Fare"].mean())
data["FareAdj"] = data["Fare"]/data["Ticket_Freq"]/222
data["FamilySize"] = (data["SibSp"] + data["Parch"] + 1) / 11
data["PclassAdj"] = data["Pclass"] / 3

data["FamilyOneSurvived"] = 0
data["FamilyAllDied"] = 0

data.head()

PassengerId  Perished  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

             Ticket     Fare Cabin Embarked  Ticket_Freq Title    Surname  \
0         A/5 21171   7.2500   NaN        S            9    Mr     Braund   
1          PC 17599  71.2833   C85        C            8   Mrs    Cumings   
2  STON/O2. 3101282   7.9250   NaN        S           16  Miss  Heikkinen   
3            113803  53.1000  C123        S            6   Mrs   Futrelle   
4            373450   8.0500   NaN        S            6    Mr      Allen   

    FareAdj  FamilySize  PclassAdj  FamilyOneSurvived  FamilyAllDied  
0  0.003629    0.181818   1.000000                  0              0  
1  0.040137    0.181818   0.333333                  0              0  
2  0.002231    0.090909   1.000000                  0              0  
3  0.039865    0.181818   0.333333                  0              0  
4  0.006044    0.090909   1.000000                  0              0

# Age予測

In [6]:
age = data[['Age', 'Pclass', 'Sex', 'Title', 'SibSp', 'Parch', 'Fare']]

age_dummies = pd.get_dummies(age)

known_age = age_dummies[age_dummies["Age"].notnull()]
null_age = age_dummies[age_dummies["Age"].isnull()]

age_X = known_age.iloc[:, 1:]
age_y = known_age.iloc[:, 0]

rf = RandomForestRegressor()
rf.fit(age_X, age_y)

pred_Age = rf.predict(null_age.iloc[:, 1:])
data.loc[(data.Age.isnull()),'Age'] = pred_Age

In [7]:
data["Ageadj"] = data["Age"] / 80

In [8]:
cols = ['Perished', 'Sex', 'Ageadj', 'FamilySize', 'FareAdj', 'FamilyOneSurvived', 'FamilyAllDied', 'PclassAdj']

In [9]:
train_df = pd.DataFrame(data[cols].iloc[:len(train)])

train = data.iloc[:len(train)]
for i in range(0, len(train_df)):
  name = train["Surname"].iloc[i]

  tmp_df = train[(train["Surname"]==name) & (train["PassengerId"] != i+1)]["Perished"]
  tmp_cnt = np.mean(list(tmp_df))

  if tmp_cnt > 0:
    train_df["FamilyOneSurvived"].iloc[i] = 1
  elif tmp_cnt == 0:
    train_df["FamilyAllDied"].iloc[i] = 1

In [10]:
test_df = pd.DataFrame(data[cols].iloc[len(train):])

train = data.iloc[:len(train)]
test = data.iloc[len(train):]

for i in range(len(test_df)):
  name = test["Surname"].iloc[i]
  tmp_df = train[train["Surname"] == name]["Perished"]
  tmp_cnt = np.mean(list(tmp_df))

  if tmp_cnt > 0:
    test_df["FamilyOneSurvived"].iloc[i] = 1
  elif tmp_cnt == 0:
    test_df["FamilyAllDied"].iloc[i] = 1

# NN

In [11]:
class Net(nn.Module):
    def __init__(self, input = 7):
        torch.manual_seed(0)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input, 128)
        self.layer2 = nn.Linear(128, 256, bias=True)
        self.layer3 = nn.Linear(256, 128, bias=True)
        self.layer4 = nn.Linear(128, 64, bias=True)
        self.layer5 = nn.Linear(64, 16, bias=True)
        self.layer6 = nn.Linear(16, 2, bias=True)

        self.dropout1 = nn.Dropout(0.10)
 
    def forward(self, x):
        x = x.float()
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)

        x = F.relu(self.layer2(x))
        x = self.dropout1(x)

        x = F.relu(self.layer3(x))
        x = self.dropout1(x)

        x = F.relu(self.layer4(x))
        x = self.dropout1(x)

        x = F.relu(self.layer5(x))

        x = self.layer6(x)
        output = F.softmax(x, dim=-1)
        return output

In [12]:
X = train_df.drop('Perished', axis=1).values
y = train_df['Perished'].values

monitor = lambda Net: all(Net.history[-1, ('train_loss_best', 'valid_loss_best')])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# CVのが必要
model = NeuralNetClassifier(
                    Net,
                    max_epochs = 100,
                    lr = 0.001,
                    warm_start = True,
                    device = device,
                    optimizer = torch.optim.AdamW,
                    iterator_train__shuffle = True,
                    batch_size = 128,
                    iterator_train__batch_size = 64,
                    iterator_valid__batch_size = 32,
                    callbacks = [Checkpoint(), EarlyStopping()],
                    train_split = CVSplit(cv = 10)
                )

model.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.6610       0.5667        0.6392     +  0.2957
      2        0.5636       0.8556        0.5113     +  0.0584
      3        0.5066       0.8444        0.4571     +  0.0625
      4        0.4570       0.8222        0.4267     +  0.0517
      5        0.4572       0.8222        0.4103     +  0.0562
      6        0.4369       0.8556        0.3967     +  0.0742
      7        0.4227       0.8444        0.3953     +  0.0761
      8        0.4118       0.7778        0.4411        0.0661
      9        0.4221       0.8556        0.3938     +  0.0655
     10        0.4071       0.8444        0.3976        0.0546
     11        0.4043       0.8444        0.4050        0.0550
     12        0.4105       0.8333        0.4027        0.0523
     13        0.4098       0.8444        0.4060        0.0639
Stopping since valid_loss has not improved in the last 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=7, out_features=128, bias=True)
    (layer2): Linear(in_features=128, out_features=256, bias=True)
    (layer3): Linear(in_features=256, out_features=128, bias=True)
    (layer4): Linear(in_features=128, out_features=64, bias=True)
    (layer5): Linear(in_features=64, out_features=16, bias=True)
    (layer6): Linear(in_features=16, out_features=2, bias=True)
    (dropout1): Dropout(p=0.1, inplace=False)
  ),
)

In [13]:
# Validation Score
valid_acc = []
for i in range(len(model.history)):
  valid_acc.append(model.history[i]["valid_acc"])

print(f'CV: accuracy = {np.mean(valid_acc)}')

CV: accuracy = 0.815873015873016


In [14]:
x_test = test_df.drop('Perished', axis=1).values
pred = model.predict(x_test)

# 提出

In [15]:
# for ensemble
submissions = pd.read_csv("/content/drive/MyDrive/GCI/titanic/Input/gender_submission.csv")
submissions["Perished"] = pred.astype('int64')
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/ensemble/NN-0.8181.csv", index=False)

# for kaggle
submissions = submissions.rename(columns={"Perished":"Survived"})
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/kaggle/NN-0.8181.csv", index=False)
# Kaggle -> 0.8181

# for GCI
submissions = submissions.rename(columns={"Survived":"Perished"})
submissions["Perished"] = submissions["Perished"].replace({1:0, 0:1})
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/gci/GCI-NN-0.8181.csv", index=False)

# 参考・引用
* https://www.kaggle.com/code/cdeotte/titanic-deep-net-0-82296
* https://qiita.com/jun40vn/items/d8a1f71fae680589e05c
* https://cpp-learning.com/skorch/#i-3